## 安装需要的第三方库

In [ ]:
!pip install mlxtend
!pip install networkx
!pip install matplotlib

You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## 从数据集中获取关联关系

### 导入相关库

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

### 准备数据

In [3]:
shopping_list = [['豆奶','莴苣'],
                 ['莴苣','尿布','葡萄酒','甜菜'],
                 ['豆奶','尿布','葡萄酒','橙汁'],
                 ['莴苣','豆奶','尿布','葡萄酒'],
                 ['莴苣','豆奶','尿布','橙汁']]  # 商店交易信息

te = TransactionEncoder()  # 使用TransactionEncoder将原始数据转为mlxtend接受的特定数据格式
df_tf = te.fit_transform(shopping_list)  # 转为布尔值的array
print(df_tf)

df = pd.DataFrame(df_tf, columns=te.columns_)  # 转为dataframe形式，将列名转化为原来的商品名
print(df)

[[False False False  True False  True]
 [ True False  True  True  True False]
 [ True  True False False  True  True]
 [ True False False  True  True  True]
 [ True  True False  True False  True]]
      尿布     橙汁     甜菜     莴苣    葡萄酒     豆奶
0  False  False  False   True  False   True
1   True  False   True   True   True  False
2   True   True  False  False   True   True
3   True  False  False   True   True   True
4   True   True  False   True  False   True


### 求频繁项集

In [4]:
# 设置最小支持度min_support=0.4求频繁项集，use_colnames表示使用商品品表示项目
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)
# 设置按照支持度从大到小排序
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)
print(frequent_itemsets)

    support       itemsets
0       0.8           (尿布)
2       0.8           (莴苣)
4       0.8           (豆奶)
6       0.6       (莴苣, 尿布)
11      0.6       (豆奶, 莴苣)
7       0.6      (葡萄酒, 尿布)
8       0.6       (豆奶, 尿布)
3       0.6          (葡萄酒)
5       0.4       (橙汁, 尿布)
1       0.4           (橙汁)
9       0.4       (豆奶, 橙汁)
10      0.4      (莴苣, 葡萄酒)
12      0.4      (豆奶, 葡萄酒)
13      0.4   (豆奶, 橙汁, 尿布)
14      0.4  (莴苣, 葡萄酒, 尿布)
15      0.4   (豆奶, 莴苣, 尿布)
16      0.4  (豆奶, 葡萄酒, 尿布)


### 求关联规则

In [5]:
# 设置使用最小置信度=0.9来求关联规则
rules = association_rules(frequent_itemsets,metric='confidence',min_threshold=0.9)
# 设置按照置信度重大到小排序
rules.sort_values(by='confidence', ascending=False, inplace=True)
print(rules[['antecedents', 'consequents', 'support', 'confidence']])

  antecedents consequents  support  confidence
0       (葡萄酒)        (尿布)      0.6         1.0
1        (橙汁)        (尿布)      0.4         1.0
2        (橙汁)        (豆奶)      0.4         1.0
3    (豆奶, 橙汁)        (尿布)      0.4         1.0
4    (橙汁, 尿布)        (豆奶)      0.4         1.0
5        (橙汁)    (豆奶, 尿布)      0.4         1.0
6   (莴苣, 葡萄酒)        (尿布)      0.4         1.0
7   (豆奶, 葡萄酒)        (尿布)      0.4         1.0


mlxtend 使用了DataFrame的方式来描述关联规则，其中：
- antecedents: 规则先导项  
- conseqents：规则后继项  
- antecedent support：规则先导项支持度  
- consequent support：规则后继项支持度  
- support：规则支持度 （前项后项并集的支持度）
- confidence：规则置信度 （规则置信度：规则支持度 / 规则先导项支持度）  
- lift：规则提升度，表示含有先导项条件下同时含有后继项的概率，与后继项总体发生的概率之比。 
- leverage：规则杠杆率，表示当先导项与后继项独立分布时，先导项与后继项一起出现的次数比预期多多少。  
- conviction：规则确信度，与提升度类似，但用差值表示。  

## 可视化关联规则

## 导入相关库

In [6]:
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx

matplotlib.rcParams['font.sans-serif'] = ['KaiTi']
matplotlib.rcParams['font.serif'] = ['KaiTi']

## 支持度-置信度散点图

In [7]:
support = rules['support'].values
confidence = rules['confidence'].values

# 因为很多规则有相同的支持度和置信度，这里给每个规则加上了随机数以显示所有的点
for i in range (len(support)):
   support[i] = support[i] + 0.0025 * (random.randint(1,10) - 5) 
   confidence[i] = confidence[i] + 0.0025 * (random.randint(1,10) - 5)
 
plt.scatter(support, confidence, marker="*")
plt.xlabel('support')
plt.ylabel('confidence') 
plt.title('Association Rules')
plt.show()

## 使用有向图可视化规则

In [8]:
def draw_graph(rules, rules_to_show):
    G1 = nx.DiGraph()
    
    color_map=[]
    N = 50
    colors = np.random.rand(N)    
    
    strs=['R0', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11']
    for i in range (rules_to_show):      
        G1.add_nodes_from(["R"+str(i)])  # 添加规则节点
        
        for a in rules.iloc[i]['antecedents']:  # 添加规则前导项
            G1.add_nodes_from([a])
            G1.add_edge(a, "R"+str(i), color=colors[i] , weight = 2)

        for c in rules.iloc[i]['consequents']:   # 添加规则后继项
            G1.add_nodes_from([c])
            G1.add_edge("R"+str(i), c, color=colors[i],  weight=2)
        
    for node in G1:
        found_a_string = False
        for item in strs: 
            if node==item:
                found_a_string = True
        if found_a_string:
            color_map.append('yellow')  # 规则节点使用黄色
        else:
            color_map.append('green')   # 项集节点使用绿色
    
    edges = G1.edges()
    colors = [G1[u][v]['color'] for u,v in edges]
    weights = [G1[u][v]['weight'] for u,v in edges]
    
    pos = nx.spring_layout(G1, k=16, scale=1)
    nx.draw(G1, pos, edges=edges, node_color = color_map, edge_color=colors, width=weights, font_size=16, with_labels=False)            
    for p in pos:  
        pos[p][1] += 0.1
    nx.draw_networkx_labels(G1, pos)
    plt.show()

    
draw_graph(rules, 1)

d:\python37\lib\site-packages\networkx\drawing\nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):
d:\python37\lib\site-packages\networkx\drawing\nx_pylab.py:585: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  and cb.iterable(edge_color) \
d:\python37\lib\site-packages\networkx\drawing\nx_pylab.py:595: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  for c in edge_color]):
d:\python37\lib\site-packages\networkx\drawing\nx_pylab.py:676: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if cb.iterable(node_size):  # many node sizes


我们使用有向图来可视化关联规则，上图中的黄色节点R0表示一条关联规则，它总是会有前向节点和后继节点。前向节点表示规则前导项，如图中的“葡萄”，前向节点和规则节点的边的箭头指向规则节点；后继节点表示规则后继项，如图中的“尿布”，规则节点和后继节点的边的箭头指向后继节点。因此上图的有向图表示着“葡萄酒-->尿布”这一关联规则。

In [9]:
draw_graph(rules, 7)